<a href="https://colab.research.google.com/github/GeonwonSeo/MBRIS-/blob/main/MBRIS_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
#!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 56 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 120 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [ ]:
#pip install tqdm
#colab 세션 유지
# function ClickConnect(){  console.log("Connnect Clicked - Start");   document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();  console.log("Connnect Clicked - End"); };setInterval(ClickConnect, 60000)​

In [ ]:
#종 이름으로 검색해서 sp.제외 후 같은 종에 대해 학명/국명 중복검사 후 있으면 가져오고 없으면 NCBI Blast 결과 가져오기
# headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0'}
# data = requests.get('https://www.mbris.kr/pub/parcel/search/parcelSearch.do?pageIndex=1&avlParc=avlParc&selSample=CUL',headers=headers)
# soup = BeautifulSoup(data.text, 'html.parser')

import requests
import time
import random
from tqdm import tqdm
from bs4 import BeautifulSoup
collectraw = []
rawdata = []
species = 'MABIK+AL'
selSearchType = '' #[]'pressedleaf' 'drying']

for k in tqdm(range(1480,2961)):#2961
  r = requests.get(f'https://www.mbris.kr/pub/parcel/search/parcelSearch.do?pageIndex={k}&avlParc=all&searchWrd={species}&selSample=&selSearchType={selSearchType}&schClassfTyp=&dnaSelect=&selKind=&mngInstiId=&pageUnit=30')
  # pip install bs4
  soup = BeautifulSoup(r.text, 'html.parser')
  time.sleep(random.uniform(2,4))
  for i in range(1,31):
    sourcenameraw = soup.select_one(f'#frm > div.subW > table > tbody > tr:nth-child({i}) > td:nth-child(1) > a')
    try:
      collectraw = [soup.select_one(f'#frm > div.subW > table > tbody > tr:nth-child({i}) > td:nth-child({j})').text for j in range(2,8)]
    except AttributeError:
      continue
    else:
      collectraw.insert(0,sourcenameraw.text)
      rawdata.append(collectraw)
fields = ["자원번호", "보유기관","유형","학명/국명","채집지","채집일","분양가능 점수"]
rows = rawdata
import csv
with open("/content/drive/MyDrive/Colab Notebooks/rawdatatest.csv", "w") as f:
  write = csv.writer(f)
  write.writerow(fields)
  write.writerows(rows)



100%|██████████| 1481/1481 [4:23:48<00:00, 10.69s/it]


In [ ]:
#학명/국명이 영어도 나오게끔
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/rawdatatest.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/rawdatatest2.csv')
df3 = pd.concat([df1,df2])
df3.to_csv('/content/drive/MyDrive/Colab Notebooks/rawdata.csv')

,Unnamed: 0,자원번호,보유기관,유형,학명/국명,채집지,채집일,분양가능 점수
0,0,MABIK AL00033001,국립해양생물자원관,표본(석엽표본),방망이가시서실,대한민국 제주특별자치도 서귀포시 성산읍,2012-08-20,\n
1,1,MABIK AL00033002,국립해양생물자원관,표본(석엽표본),방망이가시서실,대한민국 제주특별자치도 서귀포시 성산읍,2012-08-20,\n
2,2,MABIK AL00033003,국립해양생물자원관,표본(석엽표본),방망이가시서실,대한민국 제주특별자치도 서귀포시 성산읍,2012-08-20,\n
3,3,MABIK AL00061716,국립해양생물자원관,표본(석엽표본),방망이가시서실,대한민국 강원도 강릉시 사천면,2000-03-05,\n
4,4,MABIK AL00077490,국립해양생물자원관,표본(석엽표본),방망이가시서실,대한민국 충청남도 태안군 소원면,2013-07-22,\n
...,...,...,...,...,...,...,...,...
88793,44423,MABIK AL00100374,국립해양생물자원관,표본(석엽표본),반주름말,대한민국,2022-08-07,\n
88794,44424,MABIK AL00100373,국립해양생물자원관,표본(석엽표본),반주름말,대한민국,2022-08-07,\n
88795,44425,MABIK AL00100305,국립해양생물자원관,표본(석엽표본),반주름말,대한민국,2022-06-09,\n
88796,44426,MABIK AL00100288,국립해양생물자원관,표본(석엽표본),반주름말,대한민국,2022-06-10,\n


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/rawdata.csv',skiprows=0)
#df = df.drop(columns=['Unnamed: 0'])
df['유형'] = df['유형'].str.extract('([\(\)ㄱ-ㅎ가-힣A-Za-z]+)')
df['학명/국명'] = df['학명/국명'].str.extract('([\(\)ㄱ-ㅎ가-힣A-Za-z]+)')
df2 = df[df['유형'] == '표본(석엽표본)']
dfAL = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/AL DB.xlsx')

In [ ]:
dfAL =dfAL[['SGL_SGL','COL_NUM']]
dfAL.columns = ['DB수장위치','자원번호']
dfAL.to_excel('/content/drive/MyDrive/Colab Notebooks/AL DB수장위치.xlsx')

In [ ]:
#동적 크롤링
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

options = Options()
options.add_argument("--headless")
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10)
driver.get('https://www.mbris.kr/pub/parcel/search/parcelSearch.do?pageIndex=1&avlParc=all&searchWrd=MABIK+AL&selSample=&selSearchType=&schClassfTyp=&dnaSelect=&selKind=&mngInstiId=&pageUnit=30')

link_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div.subW > table > tbody > tr:nth-child(1) > td:nth-child(1) > a"))  # 링크 요소 선택자
)
link_element.click()
content_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#middle > section.pubParcel-Body > div.pop.tblList > div.cont > ul > li:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(2)"))  # 내용 요소 선택자
)
content = content_element.text
print(content)


